In [1]:
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torchvision.models as models
from tqdm import tqdm
from torchvision import transforms
from sklearn.metrics import roc_auc_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
# Load the data
df = pd.read_csv('/content/drive/My Drive/data/train data/metadata.csv')
df.head()

,date,id_coord,plume,set,lat,lon,coord_x,coord_y,path
0,20230223,id_6675,yes,train,31.528750,74.330625,24,47,images/plume/20230223_methane_mixing_ratio_id_...
1,20230103,id_2542,yes,train,35.538000,112.524000,42,37,images/plume/20230103_methane_mixing_ratio_id_...
2,20230301,id_6546,yes,train,21.060000,84.936667,58,15,images/plume/20230301_methane_mixing_ratio_id_...
3,20230225,id_6084,yes,train,26.756667,80.973333,28,62,images/plume/20230225_methane_mixing_ratio_id_...
4,20230105,id_2012,yes,train,34.800000,40.770000,59,44,images/plume/20230105_methane_mixing_ratio_id_...


In [35]:
# Split the data
train_df, valid_df = train_test_split(df, train_size =0.8, test_size=0.2, random_state=42)

In [36]:
# extract file names from path
train_df['new_path'] = train_df['path'].str.replace(r'images/(plume|no_plume)/', '', regex=True) + ".tif"
valid_df['new_path'] = valid_df['path'].str.replace(r'images/(plume|no_plume)/', '', regex=True) + ".tif"

In [37]:
#For each train and validation datasets seggregate data into separate folders based on plume or no plume
import os
import random
import shutil

# Set up paths
data_dir = '/content/drive/My Drive/data/train data/images'
train_dir= 'train'
val_dir = 'validation'

plume_dir = 'plume'
no_plume_dir = 'no_plume'

# Create validation directories
os.makedirs(os.path.join(train_dir, plume_dir), exist_ok=True)
os.makedirs(os.path.join(train_dir, no_plume_dir), exist_ok=True)
os.makedirs(os.path.join(val_dir, plume_dir), exist_ok=True)
os.makedirs(os.path.join(val_dir, no_plume_dir), exist_ok=True)

for index, row in train_df.iterrows():
    image_file = row['new_path']
    is_plume = row['plume']

    if is_plume == 'yes':
      if not os.path.exists(os.path.join(train_dir, plume_dir, image_file)):
        shutil.move(os.path.join(data_dir, plume_dir, image_file),
                os.path.join(train_dir, plume_dir))
    else:
      if not os.path.exists(os.path.join(train_dir, no_plume_dir, image_file)):
        shutil.move(os.path.join(data_dir, no_plume_dir, image_file),
                os.path.join(train_dir, no_plume_dir))

for index, row in valid_df.iterrows():
    image_file = row['new_path']
    is_plume = row['plume']

    if is_plume == 'yes':
      if not os.path.exists(os.path.join(val_dir, plume_dir, image_file)):
        shutil.move(os.path.join(data_dir, plume_dir, image_file),
                os.path.join(val_dir, plume_dir))
    else:
      if not os.path.exists(os.path.join(val_dir, no_plume_dir, image_file)):
        shutil.move(os.path.join(data_dir, no_plume_dir, image_file),
                os.path.join(val_dir, no_plume_dir))


In [38]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, augment=False):
        self.root_dir = root_dir
        self.augment = augment
        self.image_list = []
        self.labels = []
        self.augmented_images = []

        plume_images = []
        no_plume_images = []

        classes = os.listdir(root_dir)
        for class_name in classes:
            class_dir = os.path.join(root_dir, class_name)
            if os.path.isdir(class_dir):
                image_names = os.listdir(class_dir)
                tiff_image_names = [img_name for img_name in image_names if img_name.lower().endswith('.tiff') or img_name.lower().endswith('.tif')]

                if class_name == 'plume':
                    plume_images.extend([os.path.join(class_dir, img_name) for img_name in tiff_image_names])
                else:
                    no_plume_images.extend([os.path.join(class_dir, img_name) for img_name in tiff_image_names])

                self.labels.extend([1 if class_name == 'plume' else 0] * len(tiff_image_names))

        # Calculate the number of images in each class
        num_images = min(len(plume_images), len(no_plume_images))

        if augment:
            augmented_images = []
            for _ in range(num_images):
                # Randomly select plume and no plume images
                plume_image_path = random.choice(plume_images)
                no_plume_image_path = random.choice(no_plume_images)

                # Load and augment plume image
                plume_image = Image.open(plume_image_path)
                plume_image = plume_image.resize((64, 64))  # Resize to 64x64
                transform = transforms.Compose([
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomRotation(10),
                    transforms.ToTensor()
                ])
                augmented_plume_image = transform(plume_image)
                augmented_images.append(augmented_plume_image)

                # Load and augment no plume image
                no_plume_image = Image.open(no_plume_image_path)
                no_plume_image = no_plume_image.resize((64, 64))  # Resize to 64x64
                augmented_no_plume_image = transform(no_plume_image)
                augmented_images.append(augmented_no_plume_image)

            self.augmented_images = augmented_images

        self.image_list = plume_images[:num_images] + no_plume_images[:num_images]
        self.labels = [1] * num_images + [0] * num_images

        if augment:
            num_augmented_images = len(self.augmented_images)
            self.image_list.extend(self.augmented_images)
            self.labels.extend(self.labels[:num_augmented_images])

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image = self.image_list[idx]
        label = self.labels[idx]

        if isinstance(image, str):
            image = Image.open(image)
            image = image.resize((64, 64))  # Resize to 64x64

            transform = transforms.Compose([
                transforms.ToTensor()
            ])
            image = transform(image)

        return image, label

In [39]:
root_dir = "train"
val_dir = "validation"

train_dataset = CustomDataset(root_dir, augment=True)
val_dataset = CustomDataset(val_dir, augment=False)


# Define batch size for training and validation
batch_size = 32

# Create the train DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create the validation DataLoader
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Verify the train and validation data loaders
for images, labels in train_loader:
    print(f"Train batch - Images: {images.shape}, Labels: {labels.shape}")
    break

for images, labels in val_loader:
    print(f"Validation batch - Images: {images.shape}, Labels: {labels.shape}")
    break

Train batch - Images: torch.Size([32, 1, 64, 64]), Labels: torch.Size([32])
Validation batch - Images: torch.Size([32, 1, 64, 64]), Labels: torch.Size([32])


In [40]:
train_dataset

In [13]:
#We can check that our train dataset is balanced between plume and no plume
num_plumes = sum(label == 1 for label in train_dataset.labels)
num_no_plumes = sum(label == 0 for label in train_dataset.labels)

print("Number of training samples:", len(train_dataset))
print(f"Number of plumes: {num_plumes}")
print(f"Number of no plumes: {num_no_plumes}")

Number of training samples: 568
Number of plumes: 284
Number of no plumes: 284


#WideResNet50

In [10]:
# Define the device for training (CPU or GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained ResNet50 model
resnet50 = models.wide_resnet50_2(weights='Wide_ResNet50_2_Weights.IMAGENET1K_V2')

# Modify the first layer to accept single-channel grayscale images
resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modify the last fully connected layer for binary classification with softmax activation
num_classes = 2  # 2 classes: 1 or 0
resnet50.fc = nn.Sequential(
    nn.Linear(resnet50.fc.in_features, num_classes)
)

Downloading: "https://download.pytorch.org/models/wide_resnet50_2-9ba9bcbe.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet50_2-9ba9bcbe.pth
100%|██████████| 263M/263M [00:23<00:00, 11.7MB/s]


In [14]:
# Move the model to the device
resnet50 = resnet50.to(device)

# Define the loss function (criterion)
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(resnet50.parameters(), lr=0.001)

num_epochs = 50

In [15]:
best_auc = 0.0
# Training loop
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    resnet50.train()  # Set the model to training mode

    epoch_loss = 0.0  # Accumulator for epoch loss
    for images, labels in train_loader:
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device)

        # Forward pass
        outputs = resnet50(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        epoch_loss += loss.item()

    # Compute the average loss for the epoch
    epoch_loss /= len(train_loader)
    tqdm.write(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {epoch_loss}")

    # Validation loop
    resnet50.eval()  # Set the model to evaluation mode

    true_probabilities = []
    predicted_probabilities = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device, dtype=torch.float)
            labels = labels.to(device)

            # Forward pass
            outputs = resnet50(images)
            probabilities = torch.softmax(outputs, dim=1)

            # Collect predicted probabilities and true labels
            predicted_probabilities.extend(probabilities[:, 1].cpu().numpy())  # Assuming binary classification
            true_probabilities.extend(labels.cpu().numpy())

    # Compute the AUC
    auc = roc_auc_score(true_probabilities, predicted_probabilities)
    print(f"Validation AUC: {auc}")

    # Check if current accuracy is better than the previous best accuracy
    if auc > best_auc:
        best_auc = auc
        torch.save(resnet50.state_dict(), f"resnet50_{best_auc:.4f}.pth")  # Save the model with AUC in the filename
        print("Saved")

Epochs:   0%|          | 0/50 [00:02<?, ?it/s]

Epoch 1/50 - Average Loss: 0.2112888248844279
Validation AUC: 0.7691326530612245


Epochs:   2%|▏         | 1/50 [00:02<02:15,  2.77s/it]

Saved


Epochs:   4%|▍         | 2/50 [00:04<01:51,  2.33s/it]

...

Epoch 48/50 - Average Loss: 0.01630731696801983
Validation AUC: 0.7834821428571429


Epochs:  98%|█████████▊| 49/50 [01:45<00:02,  2.07s/it]

Epoch 49/50 - Average Loss: 0.015802534648527298
Validation AUC: 0.78125


Epochs: 100%|██████████| 50/50 [01:47<00:00,  2.14s/it]

Epoch 50/50 - Average Loss: 0.016014952371885254
Validation AUC: 0.7838010204081634


###SGD with momentum and LR scheduler - ❌

In [20]:
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR
optimizer = SGD(resnet50.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

In [21]:
num_epochs = 40
from sklearn.metrics import classification_report, f1_score, roc_auc_score
best_auc = 0.0
# Training loop
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    resnet50.train()  # Set the model to training mode

    epoch_loss = 0.0  # Accumulator for epoch loss
    for images, labels in train_loader:
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device)

        # Forward pass
        outputs = resnet50(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        epoch_loss += loss.item()

    # Compute the average loss for the epoch
    epoch_loss /= len(train_loader)
    tqdm.write(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {epoch_loss}")

    scheduler.step()

    # Validation loop
    resnet50.eval()  # Set the model to evaluation mode

    true_probabilities = []
    predicted_probabilities = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device, dtype=torch.float)
            labels = labels.to(device)

            # Forward pass
            outputs = resnet50(images)
            probabilities = torch.softmax(outputs, dim=1)

            # Collect predicted probabilities and true labels
            predicted_probabilities.extend(probabilities[:, 1].cpu().numpy())  # Assuming binary classification
            true_probabilities.extend(labels.cpu().numpy())

    # Compute the AUC
    auc = roc_auc_score(true_probabilities, predicted_probabilities)
    print(f"Validation AUC: {auc}")

    # Check if current accuracy is better than the previous best accuracy
    if auc > best_auc:
        best_auc = auc
        torch.save(resnet50.state_dict(), f"resnet50_{best_auc:.4f}.pth")  # Save the model with AUC in the filename
        print("Saved")

Epochs:   0%|          | 0/40 [00:01<?, ?it/s]

Epoch 1/40 - Average Loss: 0.015233904148582647
Validation AUC: 0.7815688775510204


Epochs:   2%|▎         | 1/40 [00:02<01:40,  2.57s/it]

Saved


Epochs:   5%|▌         | 2/40 [00:04<01:21,  2.15s/it]

...

Epoch 38/40 - Average Loss: 0.015324722350543298
Validation AUC: 0.7790178571428572


Epochs:  98%|█████████▊| 39/40 [01:15<00:01,  1.82s/it]

Epoch 39/40 - Average Loss: 0.015949370959560838
Validation AUC: 0.7825255102040816


Epochs: 100%|██████████| 40/40 [01:17<00:00,  1.94s/it]

Epoch 40/40 - Average Loss: 0.015427154767924852
Validation AUC: 0.7825255102040816


###AdaGrad with L2 reg - ❌

In [27]:
import torch.optim as optim
optimizer = optim.Adagrad(resnet50.parameters(), lr=0.01)#, weight_decay=1e-4)

In [28]:
num_epochs = 40
from sklearn.metrics import classification_report, f1_score, roc_auc_score
best_auc = 0.0
# Training loop
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    resnet50.train()  # Set the model to training mode

    epoch_loss = 0.0  # Accumulator for epoch loss
    for images, labels in train_loader:
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device)

        # Forward pass
        outputs = resnet50(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        epoch_loss += loss.item()

    # Compute the average loss for the epoch
    epoch_loss /= len(train_loader)
    tqdm.write(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {epoch_loss}")

    # Validation loop
    resnet50.eval()  # Set the model to evaluation mode

    true_probabilities = []
    predicted_probabilities = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device, dtype=torch.float)
            labels = labels.to(device)

            # Forward pass
            outputs = resnet50(images)
            probabilities = torch.softmax(outputs, dim=1)

            # Collect predicted probabilities and true labels
            predicted_probabilities.extend(probabilities[:, 1].cpu().numpy())  # Assuming binary classification
            true_probabilities.extend(labels.cpu().numpy())

    # Compute the AUC
    auc = roc_auc_score(true_probabilities, predicted_probabilities)
    print(f"Validation AUC: {auc}")

    # Check if current accuracy is better than the previous best accuracy
    if auc > best_auc:
        best_auc = auc
        torch.save(resnet50.state_dict(), f"resnet50_{best_auc:.4f}.pth")  # Save the model with AUC in the filename
        print("Saved")

Epochs:   0%|          | 0/40 [00:01<?, ?it/s]

Epoch 1/40 - Average Loss: 1.7121212784614828
Validation AUC: 0.5653698979591837


Epochs:   2%|▎         | 1/40 [00:02<01:39,  2.56s/it]

Saved


Epochs:   2%|▎         | 1/40 [00:04<01:39,  2.56s/it]

...

Epoch 38/40 - Average Loss: 0.0344986958662048
Validation AUC: 0.6852678571428572


Epochs:  98%|█████████▊| 39/40 [01:20<00:01,  1.99s/it]

Epoch 39/40 - Average Loss: 0.03545105261986868
Validation AUC: 0.7110969387755102


Epochs: 100%|██████████| 40/40 [01:22<00:00,  2.07s/it]

Epoch 40/40 - Average Loss: 0.03145034502570828
Validation AUC: 0.701530612244898


###RMSProp with L2 - ❌

In [29]:
optimizer = optim.RMSprop(resnet50.parameters(), lr=0.001)#, weight_decay=1e-5)

In [30]:
num_epochs = 40
from sklearn.metrics import classification_report, f1_score, roc_auc_score
best_auc = 0.0
# Training loop
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    resnet50.train()  # Set the model to training mode

    epoch_loss = 0.0  # Accumulator for epoch loss
    for images, labels in train_loader:
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device)

        # Forward pass
        outputs = resnet50(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        epoch_loss += loss.item()

    # Compute the average loss for the epoch
    epoch_loss /= len(train_loader)
    tqdm.write(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {epoch_loss}")

    # Validation loop
    resnet50.eval()  # Set the model to evaluation mode

    true_probabilities = []
    predicted_probabilities = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device, dtype=torch.float)
            labels = labels.to(device)

            # Forward pass
            outputs = resnet50(images)
            probabilities = torch.softmax(outputs, dim=1)

            # Collect predicted probabilities and true labels
            predicted_probabilities.extend(probabilities[:, 1].cpu().numpy())  # Assuming binary classification
            true_probabilities.extend(labels.cpu().numpy())

    # Compute the AUC
    auc = roc_auc_score(true_probabilities, predicted_probabilities)
    print(f"Validation AUC: {auc}")

    # Check if current accuracy is better than the previous best accuracy
    if auc > best_auc:
        best_auc = auc
        torch.save(resnet50.state_dict(), f"resnet50_{best_auc:.4f}.pth")  # Save the model with AUC in the filename
        print("Saved")

Epochs:   0%|          | 0/40 [00:01<?, ?it/s]

Epoch 1/40 - Average Loss: 1.472257886495855
Validation AUC: 0.5770089285714286


Epochs:   2%|▎         | 1/40 [00:02<01:38,  2.52s/it]

Saved


Epochs:   2%|▎         | 1/40 [00:04<01:38,  2.52s/it]

...

Epoch 38/40 - Average Loss: 0.0393452079475133
Validation AUC: 0.6935586734693877


Epochs:  98%|█████████▊| 39/40 [01:20<00:01,  1.93s/it]

Epoch 39/40 - Average Loss: 0.06899645169162089
Validation AUC: 0.6827168367346939


Epochs: 100%|██████████| 40/40 [01:22<00:00,  2.06s/it]

Epoch 40/40 - Average Loss: 0.031419607720130846
Validation AUC: 0.6836734693877551


#Resnet101

In [31]:
# Define the device for training (CPU or GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained ResNet101 model
resnet101 = models.resnet101(weights='ResNet101_Weights.DEFAULT')

# Modify the first layer to accept single-channel grayscale images
resnet101.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Modify the last fully connected layer for binary classification with softmax activation
num_classes = 2  # 2 classes: 1 or 0
resnet101.fc = nn.Sequential(
    nn.Linear(resnet101.fc.in_features, num_classes)
)

Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to /root/.cache/torch/hub/checkpoints/resnet101-cd907fc2.pth
100%|██████████| 171M/171M [00:02<00:00, 74.4MB/s]


In [32]:
# Move the model to the device
resnet101 = resnet101.to(device)

# Define the loss function (criterion)
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(resnet101.parameters(), lr=0.001)

num_epochs = 40

In [33]:
from sklearn.metrics import classification_report, f1_score, roc_auc_score
best_auc = 0.0
# Training loop
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    resnet101.train()  # Set the model to training mode

    epoch_loss = 0.0  # Accumulator for epoch loss
    for images, labels in train_loader:
        images = images.to(device, dtype=torch.float)
        labels = labels.to(device)

        # Forward pass
        outputs = resnet101(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        epoch_loss += loss.item()

    # Compute the average loss for the epoch
    epoch_loss /= len(train_loader)
    tqdm.write(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {epoch_loss}")

    # Validation loop
    resnet101.eval()  # Set the model to evaluation mode

    true_probabilities = []
    predicted_probabilities = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device, dtype=torch.float)
            labels = labels.to(device)

            # Forward pass
            outputs = resnet101(images)
            probabilities = torch.softmax(outputs, dim=1)

            # Collect predicted probabilities and true labels
            predicted_probabilities.extend(probabilities[:, 1].cpu().numpy())  # Assuming binary classification
            true_probabilities.extend(labels.cpu().numpy())

    # Compute the AUC
    auc = roc_auc_score(true_probabilities, predicted_probabilities)
    print(f"Validation AUC: {auc}")

    # Check if current accuracy is better than the previous best accuracy
    if auc > best_auc:
        best_auc = auc
        torch.save(resnet101.state_dict(), f"resnet101_{best_auc:.4f}.pth")  # Save the model with AUC in the filename
        print("Saved")

Epochs:   0%|          | 0/40 [00:02<?, ?it/s]

Epoch 1/40 - Average Loss: 0.7082766029569838
Validation AUC: 0.7133290816326531


Epochs:   2%|▎         | 1/40 [00:02<01:39,  2.56s/it]

Saved


Epochs:   5%|▌         | 2/40 [00:04<01:21,  2.15s/it]

...

Epoch 38/40 - Average Loss: 0.12047055529223548
Validation AUC: 0.8478954081632654


Epochs:  98%|█████████▊| 39/40 [01:19<00:02,  2.10s/it]

Epoch 39/40 - Average Loss: 0.11847552667475408
Validation AUC: 0.8128188775510204


Epochs: 100%|██████████| 40/40 [01:20<00:00,  2.02s/it]

Epoch 40/40 - Average Loss: 0.12672050116169783
Validation AUC: 0.7946428571428571
